In [1]:
%load_ext autoreload
%autoreload 2
import datamol as dm
from fastpdb_convertor import create_zarr_from_pdb
from pathlib import Path
from polaris.dataset import DatasetFactory
from polaris.dataset.converters import SDFConverter

In [10]:
sdfs = []
pdbs = []
for path in dm.fs.glob("/mnt/ps/home/CORP/lu.zhu/project/polaris/notebook/dev/posebusters_paper_data/posebusters_benchmark_set/5S8I_2LY/*"):
    if path.endswith('sdf'):
        sdfs.append(path.replace("file://", ""))
    elif path.endswith('pdb'):
        pdbs.append(path.replace("file://", ""))

In [11]:
sdf_converter = SDFConverter()

In [12]:
# Create a new factory object
SAVE_DIR = '/mnt/ps/home/CORP/lu.zhu/project/polaris/notebook/dev'
save_dst = dm.fs.join(SAVE_DIR, "posebusters.zarr")
factory = DatasetFactory(zarr_root_path=save_dst)
factory.reset(save_dst)

In [13]:
protein_name = Path(path).parents[0].name

file_list = ["ligand", "ligand_start_conf", "ligands"]
root = "/mnt/ps/home/CORP/lu.zhu/project/polaris/notebook/dev/posebusters_paper_data/posebusters_benchmark_set"

In [14]:
factory = DatasetFactory(zarr_root_path=save_dst)
factory.reset(save_dst)

# Process your SDF files
for file in file_list:
    sdf_path = dm.fs.join(root, protein_name, f"{protein_name}_{file}.sdf")
    factory.register_converter("sdf", SDFConverter(smiles_column=f"{file}_smiles", mol_column=file, mol_prop_as_cols=False))
    factory.add_from_file(sdf_path)

factory.register_converter("pdb", PDBConverter())
# Build the dataset
dataset = factory.build()

2024-08-07 09:02:33.629 | INFO     | polaris.dataset._factory:register_converter:112 - You are overwriting the converter for the sdf extension.
2024-08-07 09:02:33.677 | INFO     | polaris.dataset._factory:register_converter:112 - You are overwriting the converter for the sdf extension.


In [16]:
dataset.table

,ligand_smiles,ligand,ligand_start_conf_smiles,ligand_start_conf,ligands_smiles,ligands
0,CNC(=O)C1=C2OCCOC2=CS1,ligand#0,[H]C1=C2OC([H])([H])C([H])([H])OC2=C(C(=O)N([H...,ligand_start_conf#0,CNC(=O)C1=C2OCCOC2=CS1,ligands#0


In [17]:
pdbs[0]

'/mnt/ps/home/CORP/lu.zhu/project/polaris/notebook/dev/posebusters_paper_data/posebusters_benchmark_set/5S8I_2LY/5S8I_2LY_protein.pdb'

In [18]:
pdb_pointer = 0
create_zarr_from_pdb(pdb_file = pdbs[0], pdb_pointer=pdb_pointer,
                     zarr_file=dataset.zarr_root_path,
                      mode='a' )

In [19]:
dataset.table["pdb"] = f"pdb#{pdb_pointer}"

In [20]:
from polaris.dataset import ColumnAnnotation

In [21]:
dataset.annotations['pdb'] = ColumnAnnotation(is_pointer=True, modality="protein_3D", 
description=None, user_attributes={})


In [22]:
dataset

{
  "name": null,
  "description": "",
  "tags": [],
  "user_attributes": {},
  "owner": null,
  "polaris_version": "0.7.7.dev1+g404aab8",
  "default_adapters": {
    "ligand": "BYTES_TO_MOL",
    "ligand_start_conf": "BYTES_TO_MOL",
    "ligands": "BYTES_TO_MOL"
  },
  "zarr_root_path": "/mnt/ps/home/CORP/lu.zhu/project/polaris/notebook/dev/posebusters.zarr",
  "readme": "",
  "annotations": {
    "ligand_smiles": {
      "is_pointer": false,
      "modality": "MOLECULE",
      "description": null,
      "user_attributes": {},
      "dtype": "object"
    },
    "ligand": {
      "is_pointer": true,
      "modality": "MOLECULE_3D",
      "description": null,
      "user_attributes": {},
      "dtype": "object"
    },
    "ligand_start_conf_smiles": {
      "is_pointer": false,
      "modality": "MOLECULE",
      "description": null,
      "user_attributes": {},
      "dtype": "object"
    },
    "ligand_start_conf": {
      "is_pointer": true,
      "modality": "MOLECULE_3D",
      "description": null,
      "user_attributes": {},
      "dtype": "object"
    },
    "ligands_smiles": {
      "is_pointer": false,
      "modality": "MOLECULE",
      "description": null,
      "user_attributes": {},
      "dtype": "object"
    },
    "ligands": {
      "is_pointer": true,
      "modality": "MOLECULE_3D",
      "description": null,
      "user_attributes": {},
      "dtype": "object"
    },
    "pdb": {
      "is_pointer": true,
      "modality": "PROTEIN_3D",
      "description": null,
      "user_attributes": {},
      "dtype": null
    }
  },
  "source": null,
  "license": null,
  "curation_reference": null,
  "cache_dir": "/mnt/ps/home/CORP/lu.zhu/.cache/polaris/datasets/29ba189a-347a-4991-998b-db6e1312c6e3",
  "md5sum": "a2abec755ef2cf649b215e3874c15207",
  "artifact_id": null,
  "n_rows": 1,
  "n_columns": 7
}

In [ ]:
# dataset.table["pdb_name"] = Path() 

In [23]:
dataset.table

,ligand_smiles,ligand,ligand_start_conf_smiles,ligand_start_conf,ligands_smiles,ligands,pdb
0,CNC(=O)C1=C2OCCOC2=CS1,ligand#0,[H]C1=C2OC([H])([H])C([H])([H])OC2=C(C(=O)N([H...,ligand_start_conf#0,CNC(=O)C1=C2OCCOC2=CS1,ligands#0,pdb#0


In [35]:
# dataset.get_data(0, 'pdb')

In [33]:
from fastpdb_convertor import zarr_to_pdb
pdb_array = zarr_to_pdb(dataset.zarr_root_path, pdb_pointer="0")

In [34]:
pdb_array

array([
	Atom(np.array([ -8.291,  12.188, -10.882], dtype=float32), chain_id="A", res_id=1316, ins_code="", res_name="TYR", hetero=False, atom_name="N", element="N", b_factor=33.38, charge=0, occupancy=1.0),
	Atom(np.array([ -7.477,  11.014, -10.438], dtype=float32), chain_id="A", res_id=1316, ins_code="", res_name="TYR", hetero=False, atom_name="CA", element="C", b_factor=27.38, charge=0, occupancy=1.0),
	Atom(np.array([-6.244, 11.465, -9.598], dtype=float32), chain_id="A", res_id=1316, ins_code="", res_name="TYR", hetero=False, atom_name="C", element="C", b_factor=21.02, charge=0, occupancy=1.0),
	Atom(np.array([-5.194, 10.764, -9.578], dtype=float32), chain_id="A", res_id=1316, ins_code="", res_name="TYR", hetero=False, atom_name="O", element="O", b_factor=19.57, charge=0, occupancy=1.0),
	Atom(np.array([ -7.069,  10.128, -11.616], dtype=float32), chain_id="A", res_id=1316, ins_code="", res_name="TYR", hetero=False, atom_name="CB", element="C", b_factor=29.74, charge=0, occupancy=1.